In [6]:
import pandas as pd
import numpy as np
import matplotlib as mpl

# Table Visualization

https://pandas.pydata.org/pandas-docs/stable/user_guide/style.html

Styling should be performed after the data in a `DataFrame` has been processed. The `Styler` creates an HTML `<table>` and leverages CSS styling language to manipulate many parameters including colors, fonts, borders, background, etc.
    
The `DataFrame.style` attribute is a property that returns a `Styler` object.

## Formatting the Display

### Formatting Values

The `Styler` can distinguish the *display value* from the *actual value*, in both datavalues and index or columns headers. To control the display value, the text is printed in each cell as string, and we can use the `.format()` and `.format_index()` methods to manipulate this according to a format spec string or a callable that takes a single value and returns a string. It is possible to define this for the whole table, or index, or for individual columns, or MultiIndex levels.

You do not have to overwrite your DataFrame to display it how you like.

In [7]:
df = pd.DataFrame([[38.0, 2.0, 18.0, 22.0, 21, np.nan],[19, 439, 6, 452, 226,232]],
                  index=pd.Index(['Tumour (Positive)', 'Non-Tumour (Negative)'], name='Actual Label:'),
                  columns=pd.MultiIndex.from_product([['Decision Tree', 'Regression', 'Random'],['Tumour', 'Non-Tumour']], names=['Model:', 'Predicted:']))
df.style

In [2]:
weather_df = pd.DataFrame(np.random.rand(10,2)*5,
                          index=pd.date_range(start="2021-01-01", periods=10),
                          columns=["Tokyo", "Beijing"])

def rain_condition(v):
    if v < 1.75:
        return "Dry"
    elif v < 2.75:
        return "Rain"
    return "Heavy Rain"

def make_pretty(styler):
    styler.set_caption("Weather Conditions")
    styler.format(rain_condition)
    styler.format_index(lambda v: v.strftime("%A"))
    styler.background_gradient(axis=None, vmin=1, vmax=5, cmap="YlGnBu")
    return styler

weather_df

,Tokyo,Beijing
2021-01-01,4.488348,0.327287
2021-01-02,0.737329,4.216480
2021-01-03,2.694931,3.763494
2021-01-04,0.981507,1.707421
2021-01-05,3.229752,2.792247
2021-01-06,4.950977,2.279185
2021-01-07,3.393718,3.447623
2021-01-08,3.483183,2.055713
2021-01-09,1.586901,1.588792
2021-01-10,4.278537,4.207705


In [3]:
weather_df.loc["2021-01-04":"2021-01-08"].style.pipe(make_pretty)

,Tokyo,Beijing
Monday,Dry,Dry
Tuesday,Heavy Rain,Heavy Rain
Wednesday,Heavy Rain,Rain
Thursday,Heavy Rain,Heavy Rain
Friday,Heavy Rain,Rain


### Hiding Data

The index and column headers can be completely hidden from rendering.

In [14]:
s = df.style.format('{:.0f}').hide([('Random', 'Tumour'), ('Random', 'Non-Tumour')], axis="columns")
s

## Methods to Add Styles
`
There are 3 primary methods of adding custom CSS styles to Styler:

1. Using `.set_table_styles()` to control broader areas of the table with specified internal CSS.
2. Using `.set_td_classes()` to directly link either external CSS classes to your data cells or link the internal CSS classes created by `.set_table_styles()`.
3. Using the `.apply()` and `.applymap()` functions to add direct internal CSS to specific data cells.

In [15]:
cell_hover = {  # for row hover use <tr> instead of <td>
    'selector': 'td:hover',
    'props': [('background-color', '#ffffb3')]
}
index_names = {
    'selector': '.index_name',
    'props': 'font-style: italic; color: darkgrey; font-weight:normal;'
}
headers = {
    'selector': 'th:not(.index_name)',
    'props': 'background-color: #000066; color: white;'
}
s.set_table_styles([cell_hover, index_names, headers])

In [16]:
# add styles and not to override peviously added styles

s.set_table_styles([
    {'selector': 'th.col_heading', 'props': 'text-align: center;'},
    {'selector': 'th.col_heading.level0', 'props': 'font-size: 1.5em;'},
    {'selector': 'td', 'props': 'text-align: center; font-weight: bold;'},
], overwrite=False)

In [17]:
# pass a dict to apply styles to selected column

s.set_table_styles({
    ('Regression', 'Tumour'): [{'selector': 'th', 'props': 'border-left: 1px solid white'},
                               {'selector': 'td', 'props': 'border-left: 1px solid #000066'}]
}, overwrite=False, axis=0)

In [18]:
# .applymap() (elementwise): accepts a function that takes a single value 
# and returns a string with the CSS attribute-value pair

np.random.seed(0)
df2 = pd.DataFrame(np.random.randn(10,4), columns=['A','B','C','D'])

def style_negative(v, props=''):
    return props if v < 0 else None

s2 = df2.style.applymap(style_negative, props='color:red;')\
              .applymap(lambda v: 'opacity: 20%;' if (v < 0.3) and (v > -0.3) else None)
s2

,A,B,C,D
0,1.764052,0.400157,0.978738,2.240893
1,1.867558,-0.977278,0.950088,-0.151357
2,-0.103219,0.410599,0.144044,1.454274
3,0.761038,0.121675,0.443863,0.333674
4,1.494079,-0.205158,0.313068,-0.854096
5,-2.552990,0.653619,0.864436,-0.742165
6,2.269755,-1.454366,0.045759,-0.187184
7,1.532779,1.469359,0.154947,0.378163
8,-0.887786,-1.980796,-0.347912,0.156349
9,1.230291,1.202380,-0.387327,-0.302303


In [19]:
# .apply() (column-/row-/table-wise): accepts a function that takes a Series or DataFrame 
# and returns a Series, DataFrame, or numpy array with an identical shape where each element 
# is a string with a CSS attribute-value pair

def highlight_max(s, props=''):
    return np.where(s == np.nanmax(s.values), props, '')

s2.apply(highlight_max, props='color:white;background-color:darkblue', axis=0)

,A,B,C,D
0,1.764052,0.400157,0.978738,2.240893
1,1.867558,-0.977278,0.950088,-0.151357
2,-0.103219,0.410599,0.144044,1.454274
3,0.761038,0.121675,0.443863,0.333674
4,1.494079,-0.205158,0.313068,-0.854096
5,-2.552990,0.653619,0.864436,-0.742165
6,2.269755,-1.454366,0.045759,-0.187184
7,1.532779,1.469359,0.154947,0.378163
8,-0.887786,-1.980796,-0.347912,0.156349
9,1.230291,1.202380,-0.387327,-0.302303


In [20]:
# we can use the same function across the different axes, highlighting here the DataFrame maximum in purple, 
# and row maximums in pink

s2.apply(highlight_max, props='color:white;background-color:pink;', axis=1)\
  .apply(highlight_max, props='color:white;background-color:purple', axis=None)

,A,B,C,D
0,1.764052,0.400157,0.978738,2.240893
1,1.867558,-0.977278,0.950088,-0.151357
2,-0.103219,0.410599,0.144044,1.454274
3,0.761038,0.121675,0.443863,0.333674
4,1.494079,-0.205158,0.313068,-0.854096
5,-2.552990,0.653619,0.864436,-0.742165
6,2.269755,-1.454366,0.045759,-0.187184
7,1.532779,1.469359,0.154947,0.378163
8,-0.887786,-1.980796,-0.347912,0.156349
9,1.230291,1.202380,-0.387327,-0.302303


## Acting on the Index and Column Headers

Similar application is achieved for headers by using:

1. `.applymap_index()` (elementwise): accepts a function that takes a single value and returns a string with the CSS attribute-value pair
2. `.apply_index()` (level-wise): accepts a function that takes a Series and returns a Series, or numpy array with an identical shape where each element is a string with a CSS attribute-value pair. This method passes each level of your Index one-at-a-time. To style the index use axis=0 and to style the column headers use axis=1.
`

In [22]:
s2.applymap_index(lambda v: "color:pink;" if v>4 else "color:darkblue;", axis=0)
s2.apply_index(lambda s: np.where(s.isin(["A", "B"]), "color:pink;", "color:darkblue;"), axis=1)

,A,B,C,D
0,1.764052,0.400157,0.978738,2.240893
1,1.867558,-0.977278,0.950088,-0.151357
2,-0.103219,0.410599,0.144044,1.454274
3,0.761038,0.121675,0.443863,0.333674
4,1.494079,-0.205158,0.313068,-0.854096
5,-2.552990,0.653619,0.864436,-0.742165
6,2.269755,-1.454366,0.045759,-0.187184
7,1.532779,1.469359,0.154947,0.378163
8,-0.887786,-1.980796,-0.347912,0.156349
9,1.230291,1.202380,-0.387327,-0.302303


## Tooltips and Captions

Table captions can be added with the `.set_caption()` method. You can use table styles to control the CSS relevant to the caption.

Adding tooltips using the `.set_tooltips()` method in the same way you can add CSS classes to data cells by providing a string based DataFrame with intersecting indices and columns.

In [29]:
s.set_caption("Confusion matrix for multiple cancer prediction models.")\
 .set_table_styles([{
     'selector': 'caption',
     'props': 'caption-side: bottom; font-size:1.25em;'
 }], overwrite=False)

In [24]:
tt = pd.DataFrame([['This model has a very strong true positive rate',
                    "This model's total number of false negatives is too high"]],
                  index=['Tumour (Positive)'], columns=df.columns[[0,3]])
s.set_tooltips(tt, props='visibility: hidden; position: absolute; z-index: 1; border: 1px solid #000066;'
                         'background-color: white; color: #000066; font-size: 0.8em;'
                         'transform: translate(0px, -24px); padding: 0.6em; border-radius: 0.5em;')

The only thing left to do for our table is to add the highlighting borders to draw the audience attention to the tooltips. We will create internal CSS classes as before using table styles. **Setting classes always overwrites** so we need to make sure we add the previous classes.

In [26]:
s.set_table_styles([  # create internal CSS classes
    {'selector': '.border-red', 'props': 'border: 2px dashed red;'},
    {'selector': '.border-green', 'props': 'border: 2px dashed green;'},
], overwrite=False)
cell_border = pd.DataFrame([['border-green ', ' ', ' ', 'border-red '],
                           [' ', ' ', ' ', ' ']],
                          index=df.index,
                          columns=df.columns[:4])
cell_color = pd.DataFrame([['true ', 'false ', 'true ', 'false '],
                           ['false ', 'true ', 'false ', 'true ']],
                          index=df.index,
                          columns=df.columns[:4])
s.set_td_classes(cell_color + cell_border)

## Builtin Styles

Some styling functions are common enough that we’ve “built them in” to the Styler, so you don’t have to write them and apply them yourself. The current list of such functions is:

- `.highlight_null`: for use with identifying missing data
- `.highlight_min` and `.highlight_max`: for use with identifying extremeties in data
- `.highlight_between` and `.highlight_quantile`: for use with identifying classes within data
- `.background_gradient`: a flexible method for highlighting cells based on their, or other, values on a numeric scale.
- `.text_gradient`: similar method for highlighting text based on their, or other, values on a numeric scale.text_gradient: similar method for highlighting text based on their, or other, values on a numeric scale
- `.bar`: to display mini-charts within cell backgrounds.

In [30]:
df2.iloc[0,2] = np.nan
df2.iloc[4,3] = np.nan
df2.loc[:4].style.highlight_null(color='yellow')

,A,B,C,D
0,1.764052,0.400157,nan,2.240893
1,1.867558,-0.977278,0.950088,-0.151357
2,-0.103219,0.410599,0.144044,1.454274
3,0.761038,0.121675,0.443863,0.333674
4,1.494079,-0.205158,0.313068,nan


In [31]:
df2.loc[:4].style.highlight_max(axis=1, props='color:white; font-weight:bold; background-color:darkblue;')

,A,B,C,D
0,1.764052,0.400157,nan,2.240893
1,1.867558,-0.977278,0.950088,-0.151357
2,-0.103219,0.410599,0.144044,1.454274
3,0.761038,0.121675,0.443863,0.333674
4,1.494079,-0.205158,0.313068,nan


In [32]:
left = pd.Series([1.0, 0.0, 1.0], index=["A", "B", "D"])
df2.loc[:4].style.highlight_between(left=left, right=1.5, axis=1, props='color:white; background-color:purple;')

,A,B,C,D
0,1.764052,0.400157,nan,2.240893
1,1.867558,-0.977278,0.950088,-0.151357
2,-0.103219,0.410599,0.144044,1.454274
3,0.761038,0.121675,0.443863,0.333674
4,1.494079,-0.205158,0.313068,nan


In [33]:
df2.loc[:4].style.highlight_quantile(q_left=0.85, axis=None, color='yellow')

,A,B,C,D
0,1.764052,0.400157,nan,2.240893
1,1.867558,-0.977278,0.950088,-0.151357
2,-0.103219,0.410599,0.144044,1.454274
3,0.761038,0.121675,0.443863,0.333674
4,1.494079,-0.205158,0.313068,nan


In [34]:
import seaborn as sns
cm = sns.light_palette("green", as_cmap=True)

df2.style.background_gradient(cmap=cm)

,A,B,C,D
0,1.764052,0.400157,nan,2.240893
1,1.867558,-0.977278,0.950088,-0.151357
2,-0.103219,0.410599,0.144044,1.454274
3,0.761038,0.121675,0.443863,0.333674
4,1.494079,-0.205158,0.313068,nan
5,-2.552990,0.653619,0.864436,-0.742165
6,2.269755,-1.454366,0.045759,-0.187184
7,1.532779,1.469359,0.154947,0.378163
8,-0.887786,-1.980796,-0.347912,0.156349
9,1.230291,1.202380,-0.387327,-0.302303


In [35]:
df2.style.text_gradient(cmap=cm)

,A,B,C,D
0,1.764052,0.400157,nan,2.240893
1,1.867558,-0.977278,0.950088,-0.151357
2,-0.103219,0.410599,0.144044,1.454274
3,0.761038,0.121675,0.443863,0.333674
4,1.494079,-0.205158,0.313068,nan
5,-2.552990,0.653619,0.864436,-0.742165
6,2.269755,-1.454366,0.045759,-0.187184
7,1.532779,1.469359,0.154947,0.378163
8,-0.887786,-1.980796,-0.347912,0.156349
9,1.230291,1.202380,-0.387327,-0.302303


In [41]:
df2.style.bar(subset=['A', 'B'], color='#d65f5f')

,A,B,C,D
0,1.764052,0.400157,nan,2.240893
1,1.867558,-0.977278,0.950088,-0.151357
2,-0.103219,0.410599,0.144044,1.454274
3,0.761038,0.121675,0.443863,0.333674
4,1.494079,-0.205158,0.313068,nan
5,-2.552990,0.653619,0.864436,-0.742165
6,2.269755,-1.454366,0.045759,-0.187184
7,1.532779,1.469359,0.154947,0.378163
8,-0.887786,-1.980796,-0.347912,0.156349
9,1.230291,1.202380,-0.387327,-0.302303


In [42]:
df2.style.format('{:.3f}', na_rep="")\
         .bar(align=0, vmin=-2.5, vmax=2.5, cmap="bwr", height=50,
              width=60, props="width: 120px; border-right: 1px solid black;")\
         .text_gradient(cmap="bwr", vmin=-2.5, vmax=2.5)

,A,B,C,D
0,1.764,0.400,,2.241
1,1.868,-0.977,0.950,-0.151
2,-0.103,0.411,0.144,1.454
3,0.761,0.122,0.444,0.334
4,1.494,-0.205,0.313,
5,-2.553,0.654,0.864,-0.742
6,2.270,-1.454,0.046,-0.187
7,1.533,1.469,0.155,0.378
8,-0.888,-1.981,-0.348,0.156
9,1.230,1.202,-0.387,-0.302


## Set properties

Use `Styler.set_properties` when the style doesn’t actually depend on the values. This is just a simple wrapper for `.applymap` where the function returns the same properties for all cells.

In [36]:
df2.loc[:4].style.set_properties(**{'background-color': 'black',
                           'color': 'lawngreen',
                           'border-color': 'white'})

,A,B,C,D
0,1.764052,0.400157,nan,2.240893
1,1.867558,-0.977278,0.950088,-0.151357
2,-0.103219,0.410599,0.144044,1.454274
3,0.761038,0.121675,0.443863,0.333674
4,1.494079,-0.205158,0.313068,nan


## Other Fun and Useful Stuff

### Sticky Headers

In [40]:
bigdf = pd.DataFrame(np.random.randn(16, 100))
bigdf.style.set_sticky(axis="index")

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
0,-1.048553,-1.420018,-1.706270,1.950775,-0.509652,-0.438074,-1.252795,0.777490,-1.613898,-0.212740,-0.895467,0.386902,-0.510805,-1.180632,-0.028182,0.428332,0.066517,0.302472,-0.634322,-0.362741,-0.672460,-0.359553,-0.813146,-1.726283,0.177426,-0.401781,-1.630198,0.462782,-0.907298,0.051945,0.729091,0.128983,1.139401,-1.234826,0.402342,-0.684810,-0.870797,-0.578850,-0.311553,0.056165,-1.165150,0.900826,0.465662,-1.536244,1.488252,1.895889,1.178780,-0.179925,-1.070753,1.054452,-0.403177,1.222445,0.208275,0.976639,0.356366,0.706573,0.010500,1.785870,0.126912,0.401989,1.883151,-1.347759,-1.270485,0.969397,-1.173123,1.943621,-0.413619,-0.747455,1.922942,1.480515,1.867559,0.906045,-0.861226,1.910065,-0.268003,0.802456,0.947252,-0.155010,0.614079,0.922207,0.376426,-1.099401,0.298238,1.326386,-0.694568,-0.149635,-0.435154,1.849264,0.672295,0.407462,-0.769916,0.539249,-0.674333,0.031831,-0.635846,0.676433,0.576591,-0.208299,0.396007,-1.093062
1,-1.491258,0.439392,0.166673,0.635031,2.383145,0.944479,-0.912822,1.117016,-1.315907,-0.461585,-0.068242,1.713343,-0.744755,-0.826439,-0.098453,-0.663478,1.126636,-1.079932,-1.147469,-0.437820,-0.498032,1.929532,0.949421,0.087551,-1.225436,0.844363,-1.000215,-1.544771,1.188030,0.316943,0.920859,0.318728,0.856831,-0.651026,-1.034243,0.681595,-0.803410,-0.689550,-0.455533,0.017479,-0.353994,-1.374951,-0.643618,-2.223403,0.625231,-1.602058,-1.104383,0.052165,-0.739563,1.543015,-1.292857,0.267051,-0.039283,-1.168093,0.523277,-0.171546,0.771791,0.823504,2.163236,1.336528,-0.369182,-0.239379,1.099660,0.655264,0.640132,-1.616956,-0.024326,-0.738031,0.279925,-0.098150,0.910179,0.317218,0.786328,-0.466419,-0.944446,-0.410050,-0.017020,0.379152,2.259309,-0.042257,-0.955945,-0.345982,-0.463596,0.481481,-1.540797,0.063262,0.156507,0.232181,-0.597316,-0.237922,-1.424061,-0.493320,-0.542861,0.416050,-1.156182,0.781198,1.494485,-2.069985,0.426259,0.676908
2,-0.637437,-0.397272,-0.132881,-0.297791,-0.309013,-1.676004,1.152332,1.079619,-0.813364,-1.466424,0.521065,-0.575788,0.141953,-0.319328,0.691539,0.694749,-0.725597,-1.383364,-1.582938,0.610379,-1.188859,-0.506816,-0.596314,-0.052567,-1.936280,0.188779,0.523891,0.088422,-0.310886,0.097400,0.399046,-2.772593,1.955912,0.390093,-0.652409,-0.390953,0.493742,-0.116104,-2.030684,2.064493,-0.110541,1.020173,-0.692050,1.536377,0.286344,0.608844,-1.045253,1.211145,0.689818,1.301846,-0.628088,-0.481027,2.303917,-1.060016,-0.135950,1.136891,0.097725,0.582954,-0.399449,0.370056,-1.306527,1.658131,-0.118164,-0.680178,0.666383,-0.460720,-1.334258,-1.346718,0.693773,-0.159573,-0.133702,1.077744,-1.126826,-0.730678,-0.384880,0.094352,-0.042171,-0.286887,-0.061626,-0.107305,-0.719604,-0.812993,0.274516,-0.890915,-1.157355,-0.312292,-0.157667,2.256723,-0.704700,0.943261,0.747188,-1.188945,0.773253,-1.183881,-2.659172,0.606320,-1.755891,0.450934,-0.684011,1.659551
3,1.068509,-0.453386,-0.687838,-1.214077,-0.440923,-0.280355,-0.364694,0.156704,0.578521,0.349654,-0.764144,-1.437791,1.364532,-0.689449,-0.652294,-0.521189,-1.843070,-0.477974,-0.479656,0.620358,0.698457,0.003771,0.931848,0.339965,-0.015682,0.160928,-0.190653,-0.394850,-0.267734,-1.128011,0.280442,-0.993124,0.841631,-0.249459,0.049495,0.493837,0.643314,-1.570623,-0.206904,0.880179,-1.698106,0.387280,-2.255564,-1.022507,0.038631,-1.656715,-0.985511,-1.471835,1.648135,0.164228,0.567290,-0.222675,-0.353432,-1.616474,-0.291837,-0.761492,0.857924,1.141102,1.466579,0.852552,-0.598654,-1.115897,0.766663,0.356293,-1.768538,0.355482,0.814520,0.058926,-0.185054,-0.807648,-1.446535,0.800298,-0.309114,-0.233467,1.732721,0.684501,0.370825,0.142062,1.519995,1.719589,0.929505,0.582225,-2.094603,0.123722,-0.130107,0.093953,0.943046,-2.739677,-0.

### Magnify

In [43]:
def magnify():
    return [dict(selector="th",
                 props=[("font-size", "4pt")]),
            dict(selector="td",
                 props=[('padding', "0em 0em")]),
            dict(selector="th:hover",
                 props=[("font-size", "12pt")]),
            dict(selector="tr:hover td:hover",
                 props=[('max-width', '200px'),
                        ('font-size', '12pt')])
]

In [44]:
np.random.seed(25)
cmap = cmap=sns.diverging_palette(5, 250, as_cmap=True)
bigdf = pd.DataFrame(np.random.randn(20, 25)).cumsum()

bigdf.style.background_gradient(cmap, axis=1)\
    .set_properties(**{'max-width': '80px', 'font-size': '1pt'})\
    .set_caption("Hover to magnify")\
    .format(precision=2)\
    .set_table_styles(magnify())

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
0,0.23,1.03,-0.84,-0.59,-0.96,-0.22,-0.62,1.84,-2.05,0.87,-0.92,-0.23,2.15,-1.33,0.08,-1.25,1.20,-1.05,1.06,-0.42,2.29,-2.59,2.82,0.68,-1.58
1,-1.75,1.56,-1.13,-1.10,1.03,0.00,-2.46,3.45,-1.66,1.27,-0.52,-0.02,1.52,-1.09,-1.86,-1.13,-0.68,-0.81,0.35,-0.06,1.79,-2.82,2.26,0.78,0.44
2,-0.65,3.22,-1.76,0.52,2.20,-0.37,-3.00,3.73,-1.87,2.46,0.21,-0.24,-0.10,-0.78,-3.02,-0.82,-0.21,-0.23,0.86,-0.68,1.45,-4.89,3.03,1.91,0.61
3,-1.62,3.71,-2.31,0.43,4.17,-0.43,-3.86,4.16,-2.15,1.08,0.12,0.60,-0.89,0.27,-3.67,-2.71,-0.31,-1.59,1.35,-1.83,0.91,-5.80,2.81,2.11,0.28
4,-3.35,4.48,-1.86,-1.70,5.19,-1.02,-3.81,4.72,-0.72,1.08,-0.18,0.83,-0.22,-1.08,-4.27,-2.88,-0.97,-1.78,1.53,-1.80,2.21,-6.34,3.34,2.49,2.09
5,-0.84,4.23,-1.65,-2.00,5.34,-0.99,-4.13,3.94,-1.06,-0.94,1.24,0.09,-1.78,-0.11,-4.45,-0.85,-2.06,-1.35,0.80,-1.63,1.54,-6.51,2.80,2.14,3.77
6,-0.74,5.35,-2.11,-1.13,4.20,-1.85,-3.20,3.76,-3.22,-1.23,0.34,0.57,-1.82,0.54,-4.43,-1.83,-4.03,-2.62,-0.20,-4.68,1.93,-8.46,3.34,2.52,5.81
7,-0.44,4.69,-2.30,-0.21,5.93,-2.63,-1.83,5.46,-4.50,-3.16,-1.73,0.18,0.11,0.04,-5.99,-0.45,-6.20,-3.89,0.71,-3.95,0.67,-7.26,2.97,3.39,6.66
8,0.92,5.80,-3.33,-0.65,5.99,-3.19,-1.83,5.63,-3.53,-1.30,-1.61,0.82,-2.45,-0.40,-6.06,-0.52,-6.60,-3.48,-0.04,-4.60,0.51,-5.85,3.23,2.40,5.08
9,0.38,5.54,-4.49,-0.80,7.05,-2.64,-0.44,5.35,-1.96,-0.33,-0.80,0.26,-3.37,-0.82,-6.05,-2.61,-8.45,-4.45,0.41,-4.71,1.89,-6.93,2.14,3.00,5.16
